In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Input, Flatten, Reshape, ConvLSTM2D, MaxPooling2D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
# fix random seed for reproducibility
tf.random.set_seed(7)
# pip3 install tensorflow numpy pandas scikit-learn

2023-05-10 00:19:19.811638: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 00:19:20.268097: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-10 00:19:20.268129: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-10 00:19:21.744856: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df1 = pd.read_csv("Challenge_2_submission_template.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)
df1

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha)
0,1,Chau_Phu,10.542192,105.187920,WS,T,2022-04-10,1.40,NaN
1,2,Chau_Thanh,10.400189,105.331053,SA,T,2022-07-15,1.32,NaN
2,3,Chau_Phu,10.505489,105.203926,SA,D,2022-07-14,1.40,NaN
3,4,Chau_Phu,10.523520,105.138274,WS,D,2022-04-10,1.80,NaN
4,5,Thoai_Son,10.294660,105.248528,SA,T,2022-07-20,2.20,NaN
...,...,...,...,...,...,...,...,...,...
95,96,Chau_Thanh,10.435839,105.132981,SA,D,2022-07-26,1.21,NaN
96,97,Chau_Phu,10.529357,105.147388,WS,T,2022-04-10,2.00,NaN
97,98,Chau_Thanh,10.452537,105.205118,SA,T,2022-07-20,5.50,NaN
98,99,Chau_Thanh,10.394341,105.126836,SA,T,2022-07-14,4.40,NaN


In [ ]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df2 = pd.read_csv("Challenge_2_submission_template.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)
df2

In [4]:
borne = [
    (-0.0440750, 1.1379025),
    (-0.0215250, 1.1155175),
    (-0.1473925, 1.13914),
    (0.04604250, 1.2140225),
    (21762.0000, 56916.0),
    (293.0, 48423.0)
]
mini = np.array([
    -0.0440750,
    -0.0215250,
    -0.1473925,
    0.04604250,
    21762.0000,
    293.0,
])
multi = np.array([
    1.1819775, 
    1.1370425, 
    1.2865325, 
    1.16798, 
    35154.0, 
    48130.0
])

In [31]:
x1 = []
y1 = []
x2 = []
y2 = []
for i in range(len(df)):
    with open(f"inputs_lstm/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x1.append(arr)
    if df.loc[i, "Season(SA = Summer Autumn, WS = Winter Spring)"] == "SA":
        y1.append([1, 0])
    else:
        y1.append([0, 1])
for i in range(len(df2)):
    with open(f"inputs/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x1.append(arr)
    if df.loc[i, "Season(SA = Summer Autumn, WS = Winter Spring)"] == "SA":
        y1.append([1, 0])
    else:
        y1.append([0, 1])
    
x = pad_sequences(x1+x2)
y = np.array(y1+y2)
x1 = pad_sequences(x1)
y1 = np.array(y1)
# x, y = shuffle(x, y)

/tmp/ipykernel_27220/151189036.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = pad_sequences(np.array(x))


In [6]:
class_weight = {
    0: 1/np.count_nonzero(y[:, 0]==1),
    1: 1/np.count_nonzero(y[:, 1]==1)
}

In [7]:
np.count_nonzero(y[:, 0]==1)

51

In [8]:
np.count_nonzero(y[:, 1]==1)

49

In [5]:

model = Sequential()
model.add(Input(shape=(None,100, 100, 6)))
model.add(ConvLSTM2D(filters=3, kernel_size=3, return_sequences=False))

pretrained = ResNet50(
    include_top=False, 
    input_shape=model.output_shape[1:]
)
for layer in pretrained.layers:
    layer.trainable = False
    
model.add(pretrained)
model.add(MaxPooling2D())
model.add(MaxPooling2D())
model.add(Flatten())
for i in range(20):
    model.add(Dense(50))
    if (i > 0) and (i % 5 == 0):
        model.add(BatchNormalization(axis=1))
model.add(Dense(2, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_2 (ConvLSTM2D)  (None, 98, 98, 3)         984       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 2, 2048)       0         
 2D)                                                             
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 2048)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 50)               

In [32]:
"""
model = Sequential()
model.add(Input(shape=(None,100, 100, 6)))
model.add(ConvLSTM2D(filters=3, kernel_size=3, return_sequences=False))

pretrained = ResNet50(
    include_top=False, 
    input_shape=model.output_shape[1:]
)
for layer in pretrained.layers:
    layer.trainable = False
    
model.add(pretrained)
model.add(Flatten())
model.add(Dense(10))
model.add(Dense(2, activation="softmax"))
"""
model = load_model()
model.summary()

2023-05-09 01:14:12.559369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-09 01:14:12.560524: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-09 01:14:12.560963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-09 01:14:12.561281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-09 01:14:12.561381: W tensorflow/c

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 98, 98, 3)         984       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 10)                327690    
                                                                 
 dense_1 (Dense)             (None, 2)                 22        
                                                                 
Total params: 23,916,408
Trainable params: 328,696
Non-trainable params: 23,587,712
_________________________________________________________________


In [33]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
# import os
# checkpoint = ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5', monitor="val_loss", mode="min", save_best_only=True, verbose=1)
checkpoint = ModelCheckpoint(
    filepath='model/{epoch:02d}-{val_acc:.2f}.h5', 
    monitor="val_acc", mode="max", 
    save_best_only=True, verbose=1
)
callbacks = [checkpoint]

In [35]:
%%time
history = model.fit(
    x, y, 
    batch_size=16, shuffle=True, 
    validation_data=(x1, y1)
    # class_weight=class_weight,
    epochs=1_000, callbacks=callbacks
)

Epoch 1/1000


2023-05-09 01:14:18.045874: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1128000000 exceeds 10% of free system memory.
2023-05-09 01:14:27.702736: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 112800000 exceeds 10% of free system memory.
2023-05-09 01:14:27.702782: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 112800000 exceeds 10% of free system memory.
2023-05-09 01:14:28.492536: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 112800000 exceeds 10% of free system memory.
2023-05-09 01:14:28.498018: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 112800000 exceeds 10% of free system memory.


10/10 [==============================] - ETA: 0s - loss: 4.5586 - accuracy: 0.5100
Epoch 1: accuracy improved from -inf to 0.51000, saving model to model/01-0.51.h5
10/10 [==============================] - 74s 4s/step - loss: 4.5586 - accuracy: 0.5100
Epoch 2/1000
10/10 [==============================] - ETA: 0s - loss: 2.3378 - accuracy: 0.4500
Epoch 2: accuracy did not improve from 0.51000
10/10 [==============================] - 53s 5s/step - loss: 2.3378 - accuracy: 0.4500
Epoch 3/1000


KeyboardInterrupt: 

In [103]:
import time
print(time.asctime())

Tue May  9 00:50:53 2023
